# VARIABLES INSTRUMENTALES

Cette application s'inspire étroitement du chapitre : 

https://www.econometrics-with-r.org/12-ivr.html qui est lui même un suppélment R au manuel de Stock et Watson (cf. bibliographie du cours). 

On cherche à savoir quel est l'impact de l'augmentation du prix du paquet de cigarette sur la consommation. 

Pour cela on utilise la base de données CigarettesSW du package AER, et on sélectionne un sous échantillon que l'on nomme d : 

In [ ]:
install.packages("AER")
library(AER)
d <- data("CigarettesSW",package="AER")
d <- subset(CigarettesSW, year == "1995")
class(d)
head(d)

Pour le codebook, on entre ?CigarettesSW :

In [ ]:
?CigarettesSW

# 1. Régression simple

Le plus simple est de commencer par estimer un modèle de base. On estime un modèle de régression de la demande $packs$ sur le prix $price$. 

On souhaite estimer **l'élasticité** de la demande au prix. Effectuer les transformations de données qui s'imposent.

In [ ]:
d$lprice<-log(d$price)
d$lpacks<-log(d$packs)
head(d)

Estimer le modèle. Interpréter.

In [ ]:
simple<-lm(d$lpacks~d$lprice)
summary(simple)

Jusque là, nous avons estimé nos modèles linéaires par MCO sans ajsuter nos écart-types pour l'hétéroscédasticité. Sur R il est possible de calculer des écart-type robustes de White en post-estimation par la ligne de commande : **coeftest(model, vcov = vcovHC, type = "HC1")**.

Recaclulez les écart-types associés aux coefficients du modèle que nous venons d'estimer. Sont-ils plus ou moins élevés ? 

In [ ]:
coeftest(simple, vcov = vcovHC, type = "HC1")

Pourquoi le coefficient est-il biaisé selon toute probabilité ?

# 2. Instrumentation : modèle simple

La relation entre la demande d'un bien et son prix est un problème classique en économie. 

En théorie, la question est simple : si le prix augmente, la demande baisse. 
- Cependant, en retour, le prix est également affecté par la demande (causalité inverse). 
- De plus, il est également affecté par l'offre, indissociable de la demande (variable omise). Ainsi la relation entre demande et prix ne dit que peu de choses sur l'effet causal du prix sur la demande. 

Comment limiter le biais d'estimation ?

C'est une question importante en économie de la santé. En effet, imaginons que l'on veuille limiter la mortalité liée au tabac, autrement dit faire baisser la consommation de cigarettes. Certains vont proposer une augmentation du prix du paquet pour faire diminuer la demande. Il est important de connaître précisément comment la demande va répondre.

Or si je régresse la demande sur le prix j'obtiendrai une corrélation entre les deux. 

Si l'on veut connaître l'effet causal du prix sur la demande, il faut donc trouver un facteur : 
- qui affecte le prix mais qui n'est pas lié à la demande (enfin pas autrement qu'à travers le prix), 
- qui affecte le prix mais pas l'offre. 

**On réfléchit à un instrument pour le prix, et on propose la taxe à la consommation** (*sales tax* en anglais, TVA en français).

### 2.1 Instrument : création et discussion 

On crée la variable *TVA*  à partir des variables disponibles. 

In [ ]:
d$TVA<-d$taxs-d$tax

Vérifiez graphiquement que cette taxe varie bien entre les différents Etats américains.

In [ ]:
library(ggplot2)
ggplot(d, aes(x=state,y=TVA)) + geom_point()

Expliquez pourquoi cette taxe peut être un bon instrument pour le prix du paquet.

### 2.2 Intuition : Estimation par étape

Estimer la première étape (avec erreurs-types robustes). Quel signe s'attend-on à trouver ? Que penser du $R^2$ ?

In [ ]:
ivsimple_FS <- lm(lprice ~ TVA, data = d)
coeftest(ivsimple_FS, vcov = vcovHC, type = "HC1")

Prédire le prix des cigarettes que l'on nommera *instrumented_lprice*.

In [ ]:
d$instrumented_lprice <- ivsimple_FS$fitted.values

Estimer la seconde étape (avec erreurs-types robustes) et conclure. 

In [ ]:
ivsimple_SS <- lm(lpacks ~ instrumented_lprice, data = d)
coeftest(ivsimple_SS, vcov = vcovHC, type = "HC1")

Retrouver ce résultat en estimant la forme réduite. *Indice : dans le modèle simple, l'estimateur par variable instrumentale est le rapport de la forme réduite sur la première étape.*

In [ ]:
ivsimple_RF <- lm(lpacks ~ TVA, data = d)
RF_FS <- coef(ivsimple_RF)[2]/coef(ivsimple_FS)[2]
RF_FS

### 2.3 Estimation jointe des deux étapes par *ivreg*
Il existe une commande sur R intégrant les deux étapes : **ivreg()** du package **AER**. En effet les deux étapes doivent être estimées de manière jointe car autrement, le fait que le prix soit une valeur prédite par un modèle est ignoré dans l'estimation des écarts-types. On remplace **lm()** par **ivreg(y~x|z,data=...)**. On estime le modèle directement par l'estimateur des variables instrumentales (et avec erreurs-types robustes).

In [ ]:
ivsimple<-ivreg(lpacks ~ lprice | TVA , data=d)
coeftest(ivsimple, vcov = vcovHC, type = "HC1")

# 3. Estimateur des variables instrumentales avec plusieurs variables

### 3.1 Estimation

On estime le modèle complet, c'est-à-dire le modèle comprenant également des variables de contrôle exogènes. 

*Indice : la structure est ivreg(y~x+w|z+w) avec x variables endogènes, w variables exogènes, z instruments.*

In [ ]:
ivmultiple <- ivreg(lpacks ~ lprice + log(income) | log(income) + 
                    TVA, data = d)
coeftest(ivmultiple, vcov = vcovHC, type = "HC1")

### 3.2 Instrument faible : test de la pertinence

Quel est le problème si notre instrument est faible ?

Pour savoir si on a un problème d'instrument faible que calcule-t-on? Quelle est la règle de décision? 

Réaliser l'estimation et le test ( Indice : le test se réalise avec **linearHypothesis(model,"coeff=0", vcov=vcovHC, type="HC1")**. Conclure.

In [ ]:
mod_relevance <- lm(lprice ~ TVA + log(income), data=d)

In [ ]:
linearHypothesis(mod_relevance1, 
                 "TVA = 0", 
                 vcov = vcovHC, type = "HC1")

Plutôt que se fatiguer, on peut obtenir le test directement avec l'option de **summary()**: **diagnostics=TRUE**. Vérifier le résultat précédent.  

In [ ]:
summary(ivmultiple, diagnostics = TRUE, vcov = vcovHC, type = "HC1")

# 4. Estimateur des variables instrumentales avec plusieurs variables et plusieurs intruments

### 4.1 Estimation

On a un second instrument : les taxes spécifiques sur les cigarettes : *tax*. Estimer le modèle multiple avec deux instruments par l'estimateur des variables instrumentales, et comparer avec le résultat de 3.1. 

In [ ]:
ivmultiple2 <- ivreg(lpacks ~ lprice + log(income) | 
                    log(income) + TVA + tax, data = d)
coeftest(ivmultiple2, vcov = vcovHC, type = "HC1")

### 4.2 Test de suridentification : test de l'exogénéité



En quoi consiste le test de suridentification ?

Calculer la statistique de test. 

In [ ]:
sargan_reg <- lm(residuals(ivmultiple2) ~ log(income) + TVA + tax, data=d)
sargan_reg_sum <- summary(sargan_reg)
Sargan_test <- sargan_reg_sum$r.squared*nrow(d)
Sargan_test

Calculer la p-value associée (sous H0, la statistique suit une loi de Chi2 à (m-k) degrés de liberté).

In [ ]:
#rappel : une pvalue est l'aire à la droite de la valeur calculée (sous H0, proba que la stat de test soit au moins égale à ce chiffre, ou supérieure)
#degrés de liberté : m-k=nb d'instruments-nb de variabels instrumentées = 1 ici
#pchisq() donne l'aire à GAUCHE du chi2 calculé : Sargan_test
pchisq(Sargan_test,1)
#nous souhaitons la valeur à droite du chi2 calculé : 
1-pchisq(Sargan_test,1)  

Conclure. 

Vérifier que l'option **diagnostics** de **summary()** donne le même nombre (dans le tableau, c'est le 'Sargan test'). 

In [ ]:
summary(ivmultiple2, diagnostics=TRUE)

# 5. REMARQUE: Tableau de résultat : Pour produire un beau tableau

### 5.1 Installer le package stargazer

In [ ]:
install.packages("stargazer")

### 5.2 Charger la library stargazer

In [ ]:
library(stargazer)

### 5.3 Faire tourner ce code 

In [ ]:
# écarts-type robustes
rob_se <- list(sqrt(diag(vcovHC(ivsimple,    type = "HC1"))),
               sqrt(diag(vcovHC(ivmultiple,  type = "HC1"))),
               sqrt(diag(vcovHC(ivmultiple2, type = "HC1"))))

# tableau
stargazer(ivsimple, ivmultiple,ivmultiple2,
  header = FALSE, 
  type = "text",
  omit.table.layout = "n",
  digits = 3, 
  column.labels = c("Modèle simple", "Modèle multiple", "Modèle multiple <br> deux IV"),
  dep.var.labels.include = FALSE,
  dep.var.caption = "Dependent Variable: 1985-1995 Difference in Log per Pack Price",
  se = rob_se)

### Remarque : On peut créer le tableau avec **type = "html"** (que l'on peut ouvrir avec word ensuite)

<table style="text-align:center"><tr><td colspan="4" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left"></td><td colspan="3">Dependent Variable: 1985-1995 Difference in Log per Pack Price</td></tr>
<tr><td></td><td colspan="3" style="border-bottom: 1px solid black"></td></tr>
<tr><td style="text-align:left"></td><td>Modèle simple</td><td>Modèle multiple</td><td>Modèle multiple <br> deux IV</td></tr>
<tr><td style="text-align:left"></td><td>(1)</td><td>(2)</td><td>(3)</td></tr>
<tr><td colspan="4" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">lprice</td><td>-1.084<sup>***</sup></td><td>-1.003<sup>***</sup></td><td>-1.093<sup>***</sup></td></tr>
<tr><td style="text-align:left"></td><td>(0.319)</td><td>(0.322)</td><td>(0.199)</td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left">log(income)</td><td></td><td>-0.022</td><td>-0.019</td></tr>
<tr><td style="text-align:left"></td><td></td><td>(0.027)</td><td>(0.027)</td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td></tr>
<tr><td style="text-align:left">Constant</td><td>10.176<sup>***</sup></td><td>10.153<sup>***</sup></td><td>10.562<sup>***</sup></td></tr>
<tr><td style="text-align:left"></td><td>(1.663)</td><td>(1.607)</td><td>(1.044)</td></tr>
<tr><td style="text-align:left"></td><td></td><td></td><td></td></tr>
<tr><td colspan="4" style="border-bottom: 1px solid black"></td></tr><tr><td style="text-align:left">Observations</td><td>48</td><td>48</td><td>48</td></tr>
<tr><td style="text-align:left">R<sup>2</sup></td><td>0.401</td><td>0.402</td><td>0.408</td></tr>
<tr><td style="text-align:left">Adjusted R<sup>2</sup></td><td>0.388</td><td>0.376</td><td>0.382</td></tr>
<tr><td style="text-align:left">Residual Std. Error</td><td>0.190 (df = 46)</td><td>0.192 (df = 45)</td><td>0.191 (df = 45)</td></tr>
<tr><td colspan="4" style="border-bottom: 1px solid black"></td></tr></table>

On peut aussi avoir un format de tableau pour LaTex en remplaçant **type = "text"** par **type = "latex"**.